# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.float_format', '{:.3f}'.format)  # Three digits past decimal
np.set_printoptions(precision=3, suppress=True)  # Suppress scientific notation

# Set plot style
sns.set_theme(style="whitegrid")
plt.rcParams.update({'font.size': 20})
plt.rcParams['figure.figsize'] = (12, 8)

spam = pd.read_csv('https://jdgrossman.com/assets/spam.csv')

spam.sample(10)

,make,address,all,3d,our,over,remove,internet,order,mail,...,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
1509,0.000,0.560,0.560,0.000,1.120,0.560,2.250,0.000,0.000,0.560,...,0.000,0.083,0.000,0.502,0.000,0.083,16.304,148,375,1
938,0.050,0.050,0.400,0.000,0.340,0.000,0.000,0.000,0.570,0.050,...,0.019,0.099,0.000,0.089,0.079,0.009,4.913,95,1312,1
557,0.000,0.340,1.020,0.000,0.680,0.340,0.340,0.000,0.000,0.000,...,0.000,0.161,0.000,0.215,0.000,3.879,6.978,56,328,1
3976,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.476,0.000,0.000,0.000,0.000,1.285,3,18,0
2348,0.000,0.000,0.000,0.000,1.470,0.000,0.000,0.000,0.000,0.000,...,0.000,0.276,0.000,0.000,0.000,0.000,1.379,4,40,0
1603,0.340,0.170,0.170,0.000,1.380,0.690,0.170,0.170,0.000,0.170,...,0.000,0.115,0.000,0.000,0.086,0.000,4.792,38,508,1
2735,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.478,5,34,0
2584,0.000,0.000,0.910,0.000,0.000,0.000,0.000,0.450,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.320,7,103,0
1735,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.700,...,0.261,0.000,0.000,0.785,0.000,0.000,4.333,13,78,1
2865,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.267,0.133,0.133,0.000,0.000,2.607,13,73,0


## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [10]:

#1. How many emails are in the database?
print(f"Number of emails: {len(spam)}")

#2. What fraction of the emails in the database are spam?
spam_fraction = spam['is_spam'].mean()
print(f"Fraction of emails that are spam: {spam_fraction}")

# Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?
max_money_idx = spam['money'].idxmax()
max_money_pct = spam.loc[max_money_idx, 'money']
print(f"Email with highest 'money' percentage: index {max_money_idx}, percentage: {max_money_pct}")

Number of emails: 4601
Fraction of emails that are spam: 0.39404477287546186
Email with highest 'money' percentage: index 545, percentage: 12.5


####Number of emails: 4601
####Fraction of emails that are spam: 0.39404477287546186
####Email with highest 'money' percentage: index 545
####percentage: 12.5

## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [ ]:
# Your code here!



Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [ ]:
# Your code here!



## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [ ]:
# Your code here!



## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [ ]:
model = glm(is_spam ~ 1 + char_dollar + credit + money + re, family='binomial', data=spam)

summary(model)

Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [ ]:
# Your code here!

